In [247]:
# import Panda dependency
import pandas as pd

In [248]:
# filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

In [249]:
# clean student names using list, looping through, and using replace if string is found in column
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [256]:
# merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [251]:
#REQUIREMENT #1
#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

#print output
dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [273]:
#REQUIREMENT 2

#create per school summary DF
#create series for basic table
school_types = sch_df.set_index(["school_name"])["type"]

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})

school_sum_df["Total Students"] = school_sum_df["Total Students"].map("{:,}".format)
school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.0f}".format)
school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.0f}".format)
school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.0f}".format)
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)

#print output
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,3124928,"4,976","$3,124,928.00",628.0,77.0,81.0,67,82,55
Cabrera High School,1081356,"1,858","$1,081,356.00",582.0,83.1,84.0,94,97,91
Figueroa High School,1884411,"2,949","$1,884,411.00",639.0,76.7,81.2,66,81,53
Ford High School,1763916,"2,739","$1,763,916.00",644.0,77.1,80.7,68,79,54
Griffin High School,917500,"1,468","$917,500.00",625.0,83.4,83.8,93,97,91
Hernandez High School,3022020,"4,635","$3,022,020.00",652.0,77.3,80.9,67,81,54
Holden High School,248087,427,"$248,087.00",581.0,83.8,83.8,93,96,89
Huang High School,1910635,"2,917","$1,910,635.00",655.0,76.6,81.2,66,81,54
Johnson High School,3094650,"4,761","$3,094,650.00",650.0,77.1,81.0,66,81,54
Pena High School,585858,962,"$585,858.00",609.0,83.8,84.0,95,96,91
